In [1]:
%matplotlib inline
import scipy.io as io
from sklearn.metrics import (adjusted_rand_score as ari,
                             normalized_mutual_info_score as nmi)
from coclust.coclustering import CoclustInfo
from coclust.evaluation.external import accuracy
from coclust.visualization import plot_delta_kl, plot_convergence
from tqdm import tqdm
import numpy as np
import seaborn as sns
import pandas as pd

import pickle

RETRAIN = False


datasets_names = ['classic4', 'rcv4', 'ng20', 'ng5']
models_paths = ['./models/' + n +'.pkl' for n in datasets_names]
datasets_paths = ['./datasets/' + n +'.mat' for n in datasets_names]

# Retrieve the Classic4  document-term matrix from a matlab file


In [8]:
def train_coclust(n, i):
  
    model = CoclustInfo(n_row_clusters=n, n_col_clusters=n,
                        n_init=4, random_state=0, max_iter=i)
    model.fit(X)
    
    return model

In [2]:
datasets = [io.loadmat(f) for f in datasets_paths]

In [ ]:
results = {}
for idx, matlab_dict in enumerate(datasets):
    X = matlab_dict['mat'].toarray()
    true_row_labels = matlab_dict['labels'].flatten()
    n_classes = len(set(true_row_labels))
    max_iter_range = range(2, 20)
    ### entrainement des models
    models = [train_coclust(n_classes, i) for i in tqdm(max_iter_range)]
    models_dict = {i:m for i, m in zip(max_iter_range, models)}
    ### save models
    f = open(models_paths[idx], 'wb')
    pickle.dump(models_dict, f)
    f.close()
    ### recuperation des resultats
    result = pd.DataFrame(index=list(max_iter_range), columns=['nmi', 'ari', 'acc', 'criterion'])
    for index, row in result.iterrows():
        m = models_dict[index]
        predicted_row_labels = np.array(m.row_labels_)
        row['criterion'] = m.criterion
        row['nmi'] = nmi(true_row_labels, predicted_row_labels)
        row['ari'] = ari(true_row_labels, predicted_row_labels)
        row['acc'] = accuracy(true_row_labels, predicted_row_labels)
    print(result)
    results[datasets_names[idx]] = result
### save results
f = open('./results/result.pkl', 'wb')
pickle.dump(models_dict, f)

100%|██████████| 18/18 [03:28<00:00, 11.58s/it]


         nmi       ari       acc criterion
2   0.187683  0.120097  0.481675  0.170729
3    0.38187  0.269354  0.638991  0.288221
4   0.521707  0.392187   0.71779  0.354305
5    0.60374  0.451926  0.741049  0.378277
6   0.622241  0.465543  0.747251  0.381932
7   0.610181  0.454292  0.740626  0.383497
8   0.608512  0.453286  0.739216  0.384305
9   0.605918  0.450965  0.736115  0.384678
10   0.60404  0.449173  0.733437  0.384973
11  0.602311  0.448214  0.732591  0.385151
12  0.598652  0.444496  0.730195  0.385213
13  0.597305  0.442942   0.72808  0.385348
14  0.559507  0.410984  0.699887  0.385498
15  0.559538  0.411097   0.70031  0.385523
16  0.559187  0.410782   0.70031  0.385524
17  0.559562  0.411172  0.701297  0.385529
18  0.559596  0.411231  0.701438  0.385529
19  0.559596  0.411231  0.701438  0.385529


100%|██████████| 18/18 [08:34<00:00, 28.60s/it]


         nmi       ari       acc criterion
2   0.408883  0.470953  0.703617  0.355937
3   0.503983  0.537351  0.738375  0.409754
4   0.544865    0.5688  0.749804  0.426685
5   0.580634  0.598373  0.756224  0.433191
6   0.596614  0.609888  0.759668  0.435764
7   0.583637  0.603913  0.768749  0.438162
8   0.589326  0.607859  0.767966  0.438866
9   0.590612   0.60889  0.767653  0.439019
10   0.59144  0.611406  0.768906  0.439316
11  0.593969  0.613693  0.769532  0.439524
12  0.595252  0.614626  0.769532  0.439546
13  0.595252  0.614626  0.769532  0.439546
14  0.595252  0.614626  0.769532  0.439546
15  0.595252  0.614626  0.769532  0.439546
16  0.595252  0.614626  0.769532  0.439546
17  0.597447   0.60502  0.763269  0.440685
18  0.598096  0.606114  0.763895  0.441008
19    0.5993  0.607278  0.764678  0.441026


100%|██████████| 18/18 [20:34<00:00, 68.59s/it]


         nmi        ari       acc criterion
2   0.172348  0.0768351  0.185344  0.205856
3   0.302988   0.151568  0.269925  0.298907
4   0.379315   0.209318  0.331688  0.352656
5    0.43947   0.257939  0.380983  0.384309
6   0.473238   0.283744  0.402897  0.401025
7   0.493932   0.298439  0.416959  0.411502
8   0.504408    0.30788  0.425183  0.417329
9   0.516111   0.305019  0.402367  0.424122
10  0.528319   0.315058  0.410114  0.431067
11   0.53664   0.322588  0.417171  0.435258
12  0.541687   0.325549  0.419452    0.4374
13  0.545145   0.327669  0.421734  0.438763
14  0.547208   0.328229  0.424016  0.440266
15  0.547761   0.328171  0.424016  0.441825
16  0.548325   0.328687  0.425024  0.442885
17  0.549425   0.329477  0.425448  0.443862
18  0.550274   0.329925  0.425926  0.444452
19  0.551322   0.331517  0.428155  0.445059


 94%|█████████▍| 17/18 [04:28<00:20, 20.33s/it]

,nmi,ari,acc,criterion
2,0.187683,0.120097,0.481675,0.170729
